Objectif : faire plusieurs modèles de prévision, pour l'instant on va regarder ce qui se passe en classification (cheval gagne ou perd), on regardera plus tard le pt de vue régression (classement du cheval, tfaçon, sur ce dataset on a juste accès à si le cheval a gagné ou perdu)

Tous les modèles envisageables pour classification (par ordre de complexité):

 - régression logistique

- k neighbors

- decision tree

- random forest

- gradient boosting 

- SVM

- réseau de neurones

- deep learning 

In [1]:

import pandas as pd


data = pd.read_csv("dataset/export-1-pt-utf.csv")


data.head()


,place,placeoptin,rapport,cr-reunion,cr-num,cr-nb_partants,cr-Hippodrome,cr-Evt,cr-Date,cr-autostart,...,entraineurReussitePlusPlace,entraineurReussiteSpecialite,entraineurnReussiteDistance,entraineurReussiteIdcheval,chevalReussite,chevalReussitePlusPlace,chevalReussiteSpecialite,chevalnReussiteDistance,coteprobable,gains
0,0,2,0.0,1,1,17,Vincennes,0,01-01-2011,1,...,0.0,0.0,0.0,0.0,0.3877,0.3136,0.3010,0.2500,1.0000,0.8943
1,0,8,0.0,1,4,15,Vincennes,1,01-01-2011,0,...,0.0,0.0,0.0,0.0,0.5870,0.2651,0.5870,0.7142,1.0000,0.8717
2,1,1,2.4,1,6,13,Vincennes,0,01-01-2011,1,...,0.0,0.0,0.0,0.0,0.6358,0.7295,0.6358,0.5001,0.9697,0.0000
3,1,1,1.4,1,9,7,Vincennes,0,01-01-2011,1,...,0.0,0.0,0.0,0.0,0.0000,0.0096,0.0000,1.0000,1.0000,0.1892
4,0,8,0.0,1,2,15,Vincennes,0,02-01-2011,0,...,1.0,1.0,1.0,0.0,0.8642,0.8798,0.8642,0.0000,0.9211,0.8555


# Régression logistique

In [2]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19869 entries, 0 to 19868
Data columns (total 69 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   place                             19869 non-null  int64  
 1   placeoptin                        19869 non-null  int64  
 2   rapport                           19869 non-null  float64
 3   cr-reunion                        19869 non-null  int64  
 4   cr-num                            19869 non-null  int64  
 5   cr-nb_partants                    19869 non-null  int64  
 6   cr-Hippodrome                     19869 non-null  object 
 7   cr-Evt                            19869 non-null  int64  
 8   cr-Date                           19869 non-null  object 
 9   cr-autostart                      19869 non-null  int64  
 10  cr-corde                          19869 non-null  object 
 11  cr-etat du terrain                19869 non-null  object 
 12  cr-d

In [3]:
data.columns

Index(['place', 'placeoptin', 'rapport', 'cr-reunion', 'cr-num',
       'cr-nb_partants', 'cr-Hippodrome', 'cr-Evt', 'cr-Date', 'cr-autostart',
       'cr-corde', 'cr-etat du terrain', 'cr-distance', 'Numcheval',
       'CoteProbable', 'Sexe', 'NbRencontre', 'NbBattus', 'SommeNbRencontre',
       'SommeNbBattus', 'NumCourseJockey', 'QteCourseJockeyJour',
       'SumCote5Race', 'MoyenneCote5Race', 'qteCoteCheval', 'ETCote5Race',
       'SumPoids', 'SumIdJockey', 'SumidHippodrome', 'SumIdDeferre',
       'SumDistance', 'SumAllocation', 'PositionIdJockey',
       'PositionidHippodrome', 'PositionIdDeferre', 'PositionDistance',
       'PositionAllocation', 'pourcentIdJockey', 'pourcentidHippodrome',
       'pourcentIdDeferre', 'pourcentDistance', 'pourcentAllocation',
       'SommeCourse', 'nbjourdercourse', 'numeroagespec', 'numerodistancespec',
       'numerodeferrespec', 'numerosexespec', 'numeroagesexedistancespec',
       'idjockeynumcoursenbrcoursespec', 'jockeyReussite',
       'joc

pour le premier modèle, on utilise pas toutes les colonnes, on va seulement
prendre qqs features simples à interpréter i.e : 

- cote probable (cote probable du cheval avant la course)

- nbrencontre (nombre de fois où le cheval a déjà affronté au moins un des concurrents présents dans cette course.)

- nbbattus (nombre de fois où le cheval a obtenu une meilleure place qu’un concurrent dans des courses précédentes.)

- sumcote5race (somme des cotes connues du cheval sur ses 5 dernières courses)

- moyennecote5race (moyenne des cotes sur les 5 derneires courses)

- sommenbrencontre (nombre total de fois où le cheval a déjà croisé des concurrents dans le passé (somme sur tous les adversaires))

- cr-nb_partants (nombre de chevaux participants à la course)

In [4]:
features = [
    "CoteProbable",
    "NbRencontre",
    "NbBattus",
    "SumCote5Race",
    "MoyenneCote5Race",
    "SommeNbRencontre",
    "cr-nb_partants"
]

X = data[features]
y = data["place"] # 1 = gagnant, 0 = non gagnant

#on va donc vouloir prédire y à partir de X.

### Mini-nettoyage


In [5]:
print("Avant nettoyage :", X.shape)
X = X.dropna() #on supprime les lignes avec des valeurs manquantes (au moins une feature manquante)
y = y.loc[X.index]  # on garde la même forme entre X et y (on fait bien correspondre X[i] et y[i])
print("Après nettoyage :", X.shape)

Avant nettoyage : (19869, 7)
Après nettoyage : (19869, 7)


bon ben y'avait pas de valeur manquante

### Entraînement et test du modèle

In [16]:


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# on sépare le dataset 80% entraînement / 20% test (on fait pas de cross-validation pour simplifier)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)


model = LogisticRegression(max_iter=1000)

#  Entraînement
model.fit(X_train, y_train)

#  Prédictions 
y_pred = model.predict(X_test)

#  performances
accuracy = accuracy_score(y_test, y_pred) #proportion de bonnes prédictions
print(f" Accuracy : {accuracy:.3f}") #arrondi à 3 décimales

#  Matrice de confusion
print("\nMatrice de confusion :")
print(confusion_matrix(y_test, y_pred)) #affiche le nombre de TP, TN, FP, FN

#  Rapport de classification 
print("\nRapport de classification :")
print(classification_report(y_test, y_pred)) # précision(proportion de TP), recall(porportion parmi les vrais gagnants, combien ont été trouvés), f1-score (moyenne de precision et recall)


 Accuracy : 0.570

Matrice de confusion :
[[ 487 1290]
 [ 418 1779]]

Rapport de classification :
              precision    recall  f1-score   support

           0       0.54      0.27      0.36      1777
           1       0.58      0.81      0.68      2197

    accuracy                           0.57      3974
   macro avg       0.56      0.54      0.52      3974
weighted avg       0.56      0.57      0.54      3974



comme on peut le voir, on a max 58 % de précision ce qui est vraiment pas ouf, ce serait cool d'avoir au moins, disons 70% d'accuracy, ou encore mieux, de f1-score qui est encore mieux, car ça évite les prédictions qui font que prédire perdant car dans notre cas il y a 90 % de perdants, ce qui serait de la triche et trouverait bien les perdants mais mal les gagnants.

### on note que je ne me suis pour l'instant pas attaqué à : 

### --> pre-processing

### --> validation croisée 

en plus, on voit que le modèle prédit bcp mieux les 1, que les 0, surement paske ya beaucoup plus de 1 que de 0, on va essayer de régler ça

on va réduire la classe majoritaire (ici le nombre de 1)

### random undersampling

In [17]:
from sklearn.utils import resample

# on sépare les deux classes
data_majority = data[data["place"] == 1]  # gagnants
data_minority = data[data["place"] == 0]  # perdants


print(len(data_majority), len(data_minority))

# On choisit la taille minimale (pour équilibrer)
min_size = min(len(data_majority), len(data_minority))

# Échantillonnage aléatoire
data_majority_down = resample(data_majority, 
                              replace=False,     # ne pas dupliquer
                              n_samples=min_size) # même taille que minoritaire
                            

data_minority_down = resample(data_minority,
                              replace=False,
                              n_samples=min_size)

# Fusionner les deux
data_balanced = pd.concat([data_majority_down, data_minority_down])

data_balanced = data_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

#-->ici, On regroupe les deux sous-échantillons, puis on shuffle pour ne pas avoir toutes les lignes de la classe 1 puis toutes celles de la classe 0 (c’est mieux pour le split train/test)

# Vérifier la distribution
print(data_balanced["place"].value_counts())


11005 8864
place
0    8864
1    8864
Name: count, dtype: int64


puis, on fait comme taleur, mais avec le nouveau dataset équilibré en 0 et 1 

In [47]:
X = data_balanced[features]
y = data_balanced["place"]

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# on sépare le dataset 80% entraînement / 20% test (on fait pas de cross-validation pour simplifier)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)


model = LogisticRegression(max_iter=1000)

#  Entraînement
model.fit(X_train, y_train)

#  Prédictions 
y_pred = model.predict(X_test)

#  performances
accuracy = accuracy_score(y_test, y_pred) #proportion de bonnes prédictions
print(f" Accuracy : {accuracy:.3f}") #arrondi à 3 décimales

#  Matrice de confusion
print("\nMatrice de confusion :")
print(confusion_matrix(y_test, y_pred)) #affiche le nombre de TP, TN, FP, FN

#  Rapport de classification 
print("\nRapport de classification :")
print(classification_report(y_test, y_pred)) # précision(proportion de TP), recall(porportion parmi les vrais gagnants, combien ont été trouvés), f1-score (moyenne de precision et recall)


 Accuracy : 0.561

Matrice de confusion :
[[1023  728]
 [ 830  965]]

Rapport de classification :
              precision    recall  f1-score   support

           0       0.55      0.58      0.57      1751
           1       0.57      0.54      0.55      1795

    accuracy                           0.56      3546
   macro avg       0.56      0.56      0.56      3546
weighted avg       0.56      0.56      0.56      3546



toujours 56 ou 57 % d'accuracy mais au moins, le f1-score est mieux 